In [1]:
import torch
import pandas as pd

# 1. Import all necessary functions from the modules
from synthetic_dataset import generate_synthetic_dataset
from preprocessing.load import load_node_features, load_adjacency_matrix, normalize_features
from preprocessing.build_graph import build_nx_graph, build_pyg_graph
from model.gnn import GraphSAGEPredictor, seed_all
from training.train import train_node_predictor, create_directed_graph_from_scores, save_and_visualize_graph
from model.model_utils import save_node_model
from training.run_dijkstra import run_dijkstra_analysis

def main():
    """
    Main function to run the Sentry Drone GNN pipeline.
    """
    # 1.5. Generate Synthetic Dataset (Temporary)
    generate_synthetic_dataset(num_nodes=10)

    # 2. Setup & Configuration
    seed_all(42)
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    print(f"Using device: {device}")

    NODES_CSV_PATH = "data/nodes_features.csv"
    ADJ_CSV_PATH = "data/adjacency.csv"
    MODEL_SAVE_PATH = "output/node_predictor_model.pt"

    # 3. Data Loading and Preprocessing
    print("Loading and Preprocessing Data")
    node_ids, features = load_node_features(NODES_CSV_PATH)
    _, adj_matrix = load_adjacency_matrix(ADJ_CSV_PATH)
    
    # Normalize features for the model
    features_normalized = normalize_features(features)

    # Build graphs: NetworkX for structure/visualization, PyG for training
    nx_graph = build_nx_graph(node_ids, features, adj_matrix)
    pyg_graph = build_pyg_graph(node_ids, features_normalized, adj_matrix)
    
    print(f"Loaded {nx_graph.number_of_nodes()} nodes and {nx_graph.number_of_edges()} edges.")

    # 4. Model Training
    # Prepare labels for training
    # For this example, we use the 'signs_of_life_score' (column index 2) as the target priority
    node_labels = torch.tensor(features_normalized[:, 2], dtype=torch.float)

    # Initialize the node predictor model
    model = GraphSAGEPredictor(
        in_channels=features_normalized.shape[1],
        hidden_channels=64,
        num_layers=2,
        aggr="max"
    )

    # Train the model
    trained_model, history = train_node_predictor(
        model=model,
        data=pyg_graph,
        node_labels=node_labels,
        epochs=100,
        device=device
    )
    
    # 5. Save the Trained Model
    save_node_model(trained_model, path=MODEL_SAVE_PATH)

    # 6. Prediction and Directed Graph Generation
    print("\nGenerating Final Directed Rescue Graph")
    trained_model.eval()
    with torch.no_grad():
        final_node_scores = trained_model(pyg_graph.to(device)).cpu().numpy()
    
    # Create the final directed graph based on predicted scores
    final_directed_graph = create_directed_graph_from_scores(nx_graph, final_node_scores)
    
    # Visualize and save the final outputs (image and adjacency matrix)
    save_and_visualize_graph(final_directed_graph, final_node_scores, output_dir='output')

    # Visualize and save the final graph based on Dijkstra's Shortest Path for Comparison
    run_dijkstra_analysis()
    
    print("\nPipeline Finished Successfully")


if __name__ == '__main__':
    main()


Node features saved to data/nodes_features.csv
Graph is disconnected. Adding edges to connect components...
Graph is now connected.
Adjacency matrix saved to data/adjacency.csv

Synthetic dataset created successfully
Using device: cpu
Loading and Preprocessing Data
Loaded 10 nodes and 9 edges.
Starting Node Priority Model Training
Epoch   0 | Node Prediction Loss: 0.1090
Epoch  20 | Node Prediction Loss: 0.1072
Epoch  40 | Node Prediction Loss: 0.1031
Epoch  60 | Node Prediction Loss: 0.0496
Epoch  80 | Node Prediction Loss: 0.0500
Epoch  99 | Node Prediction Loss: 0.0397
Training Complete
Node predictor model saved to output/node_predictor_model.pt

Generating Final Directed Rescue Graph
Directed graph visualization saved to output/final_directed_graph.png
Adjacency matrix saved to output/final_adjacency_matrix.csv

Running Dijkstra from Source Node: 13.1199_77.5763 (Index: 0)
Dijkstra graph visualization saved to output/dijkstra_paths_graph.png

Final Path Identified (to farthest nod

In [2]:
import torch
import numpy as np
from preprocessing.load import load_node_features, load_adjacency_matrix
from preprocessing.build_graph import build_nx_graph, build_pyg_graph
from model.model_utils import load_node_model
from training.train import create_directed_graph_from_scores, save_and_visualize_graph

def get_scaler_from_training_data(training_csv_path: str):
    """
    Calculates the min and max values from the training data for normalization
    """
    _, features = load_node_features(training_csv_path)
    min_vals = features.min(axis=0)
    max_vals = features.max(axis=0)
    return min_vals, max_vals

def normalize_test_features(features: np.ndarray, min_vals: np.ndarray, max_vals: np.ndarray):
    """
    Normalizes test data using the scaler from the training data
    """
    ranges = max_vals - min_vals

    # Avoid division by zero if a feature had no range in the training data
    denom = ranges.copy()
    denom[denom == 0] = 1

    normalized = (features - min_vals) / denom

    return normalized

def predict_on_test_data():
    """
    Loads a trained model and runs predictions on new, unseen test data.
    """
    TRAINING_NODES_CSV = "data/nodes_features.csv"
    TEST_NODES_CSV = "data/test_nodes_features.csv"
    TEST_ADJ_CSV = "data/test_adjacency.csv"
    MODEL_PATH = "output/node_predictor_model.pt"
    OUTPUT_DIR = "output_test"

    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    print(f"Running Prediction on Device: {device}")

    print(f"Loading trained model from {MODEL_PATH}...")
    try:
        model = load_node_model(path=MODEL_PATH, device=device)
        model.eval() # set model to evaluation mode
    except FileNotFoundError:
        print(f"Error: Model file not found at {MODEL_PATH}. Please run the training script first.")
        return

    # Load and Preprocess Test Data
    print(f"Loading test data from {TEST_NODES_CSV}...")
    try:
        test_node_ids, test_features = load_node_features(TEST_NODES_CSV)
        _, test_adj_matrix = load_adjacency_matrix(TEST_ADJ_CSV)
    except FileNotFoundError:
        print("Error: Test data files not found. Please create them.")
        return

    # Get scaler from Training data
    print("Calculating scaler from training data...")
    min_vals, max_vals = get_scaler_from_training_data(TRAINING_NODES_CSV)
    
    # Normalize Test data using the training scaler
    print("Normalizing test data...")
    test_features_normalized = normalize_test_features(test_features, min_vals, max_vals)

    # Build Graphs for Test Data
    print("Building graphs for test data...")
    test_nx_graph = build_nx_graph(test_node_ids, test_features, test_adj_matrix)
    test_pyg_graph = build_pyg_graph(test_node_ids, test_features_normalized, test_adj_matrix)

    # Run Prediction
    print("Running model prediction...")
    with torch.no_grad():
        node_scores = model(test_pyg_graph.to(device)).cpu().numpy()

    # Generate and Save Final Output
    print("Generating and saving final rescue graph...")
    final_directed_graph = create_directed_graph_from_scores(test_nx_graph, node_scores)
    
    save_and_visualize_graph(
        final_directed_graph, 
        node_scores, 
        output_dir=OUTPUT_DIR
    )
    print(f"\nPrediction Complete. Results saved in '{OUTPUT_DIR}' folder.")


if __name__ == '__main__':
    predict_on_test_data()

Running Prediction on Device: cpu
Loading trained model from output/node_predictor_model.pt...
Node predictor model loaded from output/node_predictor_model.pt
Loading test data from data/test_nodes_features.csv...
Calculating scaler from training data...
Normalizing test data...
Building graphs for test data...
Running model prediction...
Generating and saving final rescue graph...
Directed graph visualization saved to output_test/final_directed_graph.png
Adjacency matrix saved to output_test/final_adjacency_matrix.csv

Prediction Complete. Results saved in 'output_test' folder.
